# Christian Sausa
# Jezzrel Aladdin S. Sambaan
# BSIT-4R5

## Project Assignment and Mechanics:

1. Group yourself with 2 Members.
2. Acquire text dataset from any online resources. Submit datasets here with a filename(section-member1LN-memberLN and title of your dataset). 1 group must have a unique text dataset. So therefore it is on a first-come-first-serve basis. The shared drive belongs to all the section so therefore you can view the other group's dataset. Do not use the IMDB dataset for they are our sample dataset for the lecture. 
	Requirement for your text dataset
	Minimum number of rows 5000
	Must contain textual data
	The dataset must be labeled. (Used for supervised machine learning)
3. Perform the following in your text datasets.
	Text Normalization process (data cleaning, tokenization, data normalization)
	Text Classification (use only 1 model, conduct training and testing, display prediction and performance evaluation)
4. Create a powerpoint presentation discussion about the conduct of text classification and upload here. filename structure (section-member1LN-member2-LN)
5. The earlier the submission of the dataset, the better. Submission of dataset will be on December 20, 2022)
6. There will be a schedule for presentation of your output on January 9-13, 2023.

In [1]:
# Import required modules
import re
import sys
import nltk
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Iterable, Counter
from nltk.corpus import wordnet, stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from google.colab import drive
drive.mount('/content/drive')
!{sys.executable} -m pip install autocorrect
!{sys.executable} -m pip install scikit-multilearn
from autocorrect import Speller
checkSpelling = Speller(lang='en')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

from numpy import mean, std
from sklearn import metrics, preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, make_scorer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from skmultilearn.problem_transform import LabelPowerset

<ipython-input-1-dcf06d7d204c>:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable, Counter


Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 KB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622381 sha256=c3c1637154445f74da7662f311e8233f7cfba92189af82cb06274ce51ed8e672
  Stored in directory: /root/.cache/pip/wheels/72/b8/3b/a90246d13090e85394a8a44b78c8abf577c0766f29d6543c75
Successfully built autocorrect
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 KB 4.1 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 1) Load the dataset

In [2]:
# Load the above dataset
dt = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/4R5-Sambaan-Sausa_Yelp-Reviews.csv')

In [3]:
# Display the head of the data
dt.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [100]:
# Split the dataset into x = independent variables, y = dependent/target variables
# the text column was assigned to x, and 3 labels on the y
x = dt.drop(['business_id', 'date', 'review_id', 'stars', 'type', 'user_id', 'cool', 'useful', 'funny'], axis=1)
y = dt[['cool', 'useful', 'funny']]

In [45]:
print("Shape of x: ", x.shape)
print("Shape of y: ", y.shape)

Shape of x:  (10000, 1)
Shape of y:  (10000, 3)


In [5]:
# Displays the x/independent variable(s)
x.head()

,text
0,My wife took me here on my birthday for breakf...
1,I have no idea why some people give bad review...
2,love the gyro plate. Rice is so good and I als...
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!..."
4,General Manager Scott Petello is a good egg!!!...


In [6]:
# Displays the y/dependent variable(s)
y.head()

,cool,useful,funny
0,2,5,0
1,0,0,0
2,0,1,0
3,1,2,0
4,0,0,0


In [7]:
# This function will fix the nested lists on the data
# for the abomination that will happen throughout this notebook
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

## 2) Data cleaning

In [8]:
# Displays the sum of all missing values
dt.isna().sum()

business_id    0
date           0
review_id      0
stars          0
text           0
type           0
user_id        0
cool           0
useful         0
funny          0
dtype: int64

## 3) Data tokenization

In [9]:
# This function tokenizes the data by words
def tokenizeWord(old_data):
    new_data = nltk.word_tokenize(old_data)
    return new_data

In [13]:
# Displays a row of the data
print(x['text'][0])
print("Type:", type(x['text'][0]))

My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.

Do yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I've ever had.  I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.

While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I've ever had.

Anyway, I can't wait to go back!
Type: <class 'str'>


In [14]:
# This for loop will call the function for word tokenization
x_words = x
for i in range(len(x)):
    x_words['text'][i] = tokenizeWord(x_words['text'][i])

In [16]:
# Displays a row of the data
print(x_words['text'][0])
print("Type:", type(x_words['text'][0]))

['My', 'wife', 'took', 'me', 'here', 'on', 'my', 'birthday', 'for', 'breakfast', 'and', 'it', 'was', 'excellent', '.', 'The', 'weather', 'was', 'perfect', 'which', 'made', 'sitting', 'outside', 'overlooking', 'their', 'grounds', 'an', 'absolute', 'pleasure', '.', 'Our', 'waitress', 'was', 'excellent', 'and', 'our', 'food', 'arrived', 'quickly', 'on', 'the', 'semi-busy', 'Saturday', 'morning', '.', 'It', 'looked', 'like', 'the', 'place', 'fills', 'up', 'pretty', 'quickly', 'so', 'the', 'earlier', 'you', 'get', 'here', 'the', 'better', '.', 'Do', 'yourself', 'a', 'favor', 'and', 'get', 'their', 'Bloody', 'Mary', '.', 'It', 'was', 'phenomenal', 'and', 'simply', 'the', 'best', 'I', "'ve", 'ever', 'had', '.', 'I', "'m", 'pretty', 'sure', 'they', 'only', 'use', 'ingredients', 'from', 'their', 'garden', 'and', 'blend', 'them', 'fresh', 'when', 'you', 'order', 'it', '.', 'It', 'was', 'amazing', '.', 'While', 'EVERYTHING', 'on', 'the', 'menu', 'looks', 'excellent', ',', 'I', 'had', 'the', 'whit

In [17]:
# Displays the tokenized dataframe
x_words

,text
0,"[My, wife, took, me, here, on, my, birthday, f..."
1,"[I, have, no, idea, why, some, people, give, b..."
2,"[love, the, gyro, plate, ., Rice, is, so, good..."
3,"[Rosie, ,, Dakota, ,, and, I, LOVE, Chaparral,..."
4,"[General, Manager, Scott, Petello, is, a, good..."
...,...
9995,"[First, visit, ..., Had, lunch, here, today, -..."
9996,"[Should, be, called, house, of, deliciousness,..."
9997,"[I, recently, visited, Olive, and, Ivy, for, b..."
9998,"[My, nephew, just, moved, to, Scottsdale, rece..."


## 4) Data normalization

### Remove punctuations

In [18]:
# This function will remove punctuations
def remove_punct(my_str):
    punctuations = '''!()-[]{};:'.'"\,<>./?@#$%^-&_0123456789~*'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [19]:
# Displays a punctuation on a row
print(x_words['text'][0][-1])
print("Type:", type(x_words['text'][0][-1]))

!
Type: <class 'str'>


In [20]:
# This for loop calls the remove punctuation function
# for each iterable on each element on the data
x_nopunct = x_words
for i in range(len(x_nopunct['text'])):
    for j in range(len(x_nopunct['text'][i])):
        x_nopunct['text'][i][j] = remove_punct(x_nopunct['text'][i][j])

In [21]:
# Displays the cleaned index of the punctuation
print(x_nopunct['text'][0][-1])
print("Type:", type(x_nopunct['text'][0][-1]))


Type: <class 'str'>


In [22]:
# Displays the dataframe cleaned from punctuations
x_nopunct

,text
0,"[My, wife, took, me, here, on, my, birthday, f..."
1,"[I, have, no, idea, why, some, people, give, b..."
2,"[love, the, gyro, plate, , Rice, is, so, good,..."
3,"[Rosie, , Dakota, , and, I, LOVE, Chaparral, D..."
4,"[General, Manager, Scott, Petello, is, a, good..."
...,...
9995,"[First, visit, , Had, lunch, here, today, , us..."
9996,"[Should, be, called, house, of, deliciousness,..."
9997,"[I, recently, visited, Olive, and, Ivy, for, b..."
9998,"[My, nephew, just, moved, to, Scottsdale, rece..."


### Remove repeated characters

In [23]:
# This function will remove repeated characters
def removeRepeatedCharacters(tokens):
    repeatPattern = re.compile(r'(\w*)(\w)\2(\w*)')
    matchSubstitution = r'\1\2\3'
    def replace(oldWord):
        if wordnet.synsets(oldWord):
            return(oldWord)
        newWord = repeatPattern.sub(matchSubstitution, oldWord)
        return replace(newWord) if newWord!= oldWord else newWord
    correctTokens = [replace(word) for word in tokens]
    return correctTokens

word = x_nopunct['text'][777][24]
sampleWord = tokenizeWord(word)
print("Original word:\t", sampleWord)
print("Corrected word:\t", removeRepeatedCharacters(sampleWord))

Original word:	 ['sooooooo']
Corrected word:	 ['so']


In [25]:
# Displays a word with repeated characters on a row
print(x_nopunct['text'][777][24])
print("Type:", type(x_nopunct['text'][777][24]))

sooooooo
Type: <class 'str'>


In [26]:
# This for loop calls the remove repeated characters function
# for each iterable on each element on the data
x_norep = x_nopunct
for i in range(len(x_norep['text'])):
    for j in range(len(x_norep['text'][i])):
        x_norep['text'][i][j] = removeRepeatedCharacters(tokenizeWord(x_norep['text'][i][j]))

In [27]:
# This for loop calls the flatten function
# for each iterable on each element on the data
# to fix the nested list
x_flat = x_norep
for i in range(len(x_flat['text'])):
    x_flat['text'][i] = list(flatten(x_flat['text'][i]))

In [29]:
# Displays the word with no repeated characters on a row after
print(x_flat['text'][777][24])
print("Type:", type(x_flat['text'][777][24]))

so
Type: <class 'str'>


### Remove contractions

In [31]:
CONTRACTION_MAP = {"ain't": "is not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have", "so's": "so as", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [32]:
# This function will remove contractions
def expand_contractions(text, contraction_mapping = CONTRACTION_MAP):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())   
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [33]:
# This for loop calls the remove contractions function
# for each iterable on each element on the data
x_nocon = x_flat
for i in range(len(x_nocon['text'])):
    for j in range(len(x_nocon['text'][i])):
        x_nocon['text'][i][j] = expand_contractions(x_nocon['text'][i][j])

In [34]:
# Displays the dataframe cleaned from contractions
x_nocon

,text
0,"[My, wife, took, me, here, on, my, birthday, f..."
1,"[I, have, no, idea, why, some, people, give, b..."
2,"[love, the, gyro, plate, Rice, is, so, good, a..."
3,"[Rosie, Dakota, and, I, LOVE, Chaparral, Dog, ..."
4,"[General, Manager, Scott, Petelo, is, a, good,..."
...,...
9995,"[First, visit, Had, lunch, here, today, used, ..."
9996,"[Should, be, called, house, of, deliciousness,..."
9997,"[I, recently, visited, Olive, and, Ivy, for, b..."
9998,"[My, nephew, just, moved, to, Scotsdale, recen..."


### Remove stopwords

In [35]:
# This function will remove stopwords
myWordDictionary = []
def remove_stopwords_orig(text):
    data = text
    stopWords = set(stopwords.words('english'))
    words = data
    
    wordsFiltered = []
    for w in words:
        if w not in stopWords:
            myWordDictionary.append(w)
            wordsFiltered.append(w)
            wordsFiltered.append('')
    return "".join(wordsFiltered)

In [37]:
# Displays a row with contraptions
print(x_nocon['text'][0])
print(x_nocon['text'][0][:10])
print("Type:", type(x_nocon['text'][0]))

['My', 'wife', 'took', 'me', 'here', 'on', 'my', 'birthday', 'for', 'breakfast', 'and', 'it', 'was', 'excellent', 'The', 'weather', 'was', 'perfect', 'which', 'made', 'sitting', 'outside', 'overlooking', 'their', 'grounds', 'an', 'absolute', 'pleasure', 'Our', 'waitress', 'was', 'excellent', 'and', 'our', 'food', 'arrived', 'quickly', 'on', 'the', 'semibusy', 'Saturday', 'morning', 'It', 'looked', 'like', 'the', 'place', 'fills', 'up', 'pretty', 'quickly', 'so', 'the', 'earlier', 'you', 'get', 'here', 'the', 'better', 'Do', 'yourself', 'a', 'favor', 'and', 'get', 'their', 'Bloody', 'Mary', 'It', 'was', 'phenomenal', 'and', 'simply', 'the', 'best', 'I', 've', 'ever', 'had', 'I', 'm', 'pretty', 'sure', 'they', 'only', 'use', 'ingredients', 'from', 'their', 'garden', 'and', 'blend', 'them', 'fresh', 'when', 'you', 'order', 'it', 'It', 'was', 'amazing', 'While', 'EVERYTHING', 'on', 'the', 'menu', 'looks', 'excellent', 'I', 'had', 'the', 'white', 'truffle', 'scrambled', 'eggs', 'vegetable',

In [38]:
# This for loop calls the remove stopwords function
# for each iterable on each element on the data
x_nostop = x_nocon
for i in range(len(x_nostop['text'])):
    for j in range(len(x_nostop['text'][i])):
        x_nostop['text'][i][j] = remove_stopwords_orig(tokenizeWord(x_nostop['text'][i][j]))

In [39]:
# Displays the row cleared from stopwords
print(x_nostop['text'][0])
print(x_nostop['text'][0][:10])
print("Type:", type(x_nostop['text'][0]))

['My', 'wife', 'took', '', '', '', '', 'birthday', '', 'breakfast', '', '', '', 'excellent', 'The', 'weather', '', 'perfect', '', 'made', 'sitting', 'outside', 'overlooking', '', 'grounds', '', 'absolute', 'pleasure', 'Our', 'waitress', '', 'excellent', '', '', 'food', 'arrived', 'quickly', '', '', 'semibusy', 'Saturday', 'morning', 'It', 'looked', 'like', '', 'place', 'fills', '', 'pretty', 'quickly', '', '', 'earlier', '', 'get', '', '', 'better', 'Do', '', '', 'favor', '', 'get', '', 'Bloody', 'Mary', 'It', '', 'phenomenal', '', 'simply', '', 'best', 'I', '', 'ever', '', 'I', '', 'pretty', 'sure', '', '', 'use', 'ingredients', '', '', 'garden', '', 'blend', '', 'fresh', '', '', 'order', '', 'It', '', 'amazing', 'While', 'EVERYTHING', '', '', 'menu', 'looks', 'excellent', 'I', '', '', 'white', 'truffle', 'scrambled', 'eggs', 'vegetable', 'skillet', '', '', '', 'tasty', '', 'delicious', 'It', 'came', '', 'pieces', '', '', 'griddled', 'bread', '', '', 'amazing', '', '', 'absolutely', '

In [40]:
# Displays the dataframe cleaned from stopwords
x_nostop

,text
0,"[My, wife, took, , , , , birthday, , breakfast..."
1,"[I, , , idea, , , people, give, bad, reviews, ..."
2,"[love, , gyro, plate, Rice, , , good, , I, als..."
3,"[Rosie, Dakota, , I, LOVE, Chaparral, Dog, Par..."
4,"[General, Manager, Scott, Petelo, , , good, eg..."
...,...
9995,"[First, visit, Had, lunch, , today, used, , Gr..."
9996,"[Should, , called, house, , deliciousness, I, ..."
9997,"[I, recently, visited, Olive, , Ivy, , busines..."
9998,"[My, nephew, , moved, , Scotsdale, recently, ,..."


### Check spelling

In [41]:
# Displays a word with incorrect/missing spelling on a row
print(x_nostop['text'][777][1])
print("Type:", type(x_nostop['text'][777][1]))

alread
Type: <class 'str'>


In [42]:
# This for loop checks the spelling
# for each iterable on each element on the data
# NOTE: this cell might take an hour and half to finish huhuhu
# depending on the powah of yah computah!
x_spell = x_nostop
for i in range(len(x_spell['text'])):
    for j in range(len(x_spell['text'][i])):
        x_spell['text'][i][j] = checkSpelling(x_spell['text'][i][j])

In [43]:
# Displays the word with corrected spelling on a row
print(x_nostop['text'][777][1])
print("Type:", type(x_nostop['text'][777][1]))

already
Type: <class 'str'>


In [44]:
# Displays the datafrom with checked spellings
x_spell

,text
0,"[My, wife, took, , , , , birthday, , breakfast..."
1,"[I, , , idea, , , people, give, bad, reviews, ..."
2,"[love, , gro, plate, Rice, , , good, , I, also..."
3,"[Rose, Dakota, , I, VE, Chaparral, Dog, Park, ..."
4,"[General, Manager, Scott, Peter, , , good, egg..."
...,...
9995,"[First, visit, Had, lunch, , today, used, , Gr..."
9996,"[Should, , called, house, , deliciousness, I, ..."
9997,"[I, recently, visited, Olive, , Ivy, , busines..."
9998,"[My, nephew, , moved, , Scottsdale, recently, ..."


### Stemming
### FOR THIS DATASET LEMMATIZING IS MORE ACCURATE THAN STEMMING
### THEREFORE STEMMING WILL BE SKIPPED

In [46]:
# Creates the porterstemmer instance
# ps = PorterStemmer()

In [47]:
# This for loop will stem every word
# for each iterable on each element on the data
# x_stem = x_spell
# for i in range(len(x_stem['text'])):
#     for j in range(len(x_stem['text'][i])):
#         x_stem['text'][i][j] = ps.stem(x_stem['text'][i][j])

In [48]:
# x_stem

### Lemmatizing

In [49]:
# Creates the wordnetlemmatizer instance
wlm = WordNetLemmatizer()

In [50]:
# Displays a word on a row that needs lemmatizing
print(x_spell['text'][9998][14])
print("Type:", type(x_spell['text'][9998][14]))

brought
Type: <class 'str'>


In [51]:
# This for loop will lemmatize every word
# for each iterable on each element on the data
x_lema = x_spell
for i in range(len(x_lema['text'])):
    for j in range(len(x_lema['text'][i])):
        x_lema['text'][i][j] = wlm.lemmatize(x_lema['text'][i][j], 'v')

In [52]:
# Displays the word after lemmatization
print(x_lema['text'][9998][14])
print("Type:", type(x_lema['text'][9998][14]))

bring
Type: <class 'str'>


In [53]:
# Displays the dataframe with lemmatization
x_lema

,text
0,"[My, wife, take, , , , , birthday, , breakfast..."
1,"[I, , , idea, , , people, give, bad, review, ,..."
2,"[love, , gro, plate, Rice, , , good, , I, also..."
3,"[Rose, Dakota, , I, VE, Chaparral, Dog, Park, ..."
4,"[General, Manager, Scott, Peter, , , good, egg..."
...,...
9995,"[First, visit, Had, lunch, , today, use, , Gro..."
9996,"[Should, , call, house, , deliciousness, I, co..."
9997,"[I, recently, visit, Olive, , Ivy, , business,..."
9998,"[My, nephew, , move, , Scottsdale, recently, ,..."


### Removing null indexes

In [55]:
# Displays a row that have null indexes
print(x_lema['text'][0])
print("Type:", type(x_lema['text'][0]))

['My', 'wife', 'take', '', '', '', '', 'birthday', '', 'breakfast', '', '', '', 'excellent', 'The', 'weather', '', 'perfect', '', 'make', 'sit', 'outside', 'overlook', '', 'ground', '', 'absolute', 'pleasure', 'Our', 'waitress', '', 'excellent', '', '', 'food', 'arrive', 'quickly', '', '', 'semibusy', 'Saturday', 'morning', 'It', 'look', 'like', '', 'place', 'fill', '', 'pretty', 'quickly', '', '', 'earlier', '', 'get', '', '', 'better', 'Do', '', '', 'favor', '', 'get', '', 'Bloody', 'Mary', 'It', '', 'phenomenal', '', 'simply', '', 'best', 'I', '', 'ever', '', 'I', '', 'pretty', 'sure', '', '', 'use', 'ingredients', '', '', 'garden', '', 'blend', '', 'fresh', '', '', 'order', '', 'It', '', 'amaze', 'While', 'EVERYTHING', '', '', 'menu', 'look', 'excellent', 'I', '', '', 'white', 'shuffle', 'scramble', 'egg', 'vegetable', 'skilled', '', '', '', 'tasty', '', 'delicious', 'It', 'come', '', 'piece', '', '', 'riddle', 'bread', '', '', 'amaze', '', '', 'absolutely', 'make', '', 'meal', 'co

In [56]:
# This for loop removes all null indexes
# for each iterable on each element on the data
x_rem = x_lema
for i in range(len(x_rem['text'])):
    x_rem['text'][i] = ' '.join(filter(None, x_rem['text'][i]))

In [57]:
# Displays a row after cleaning null indexes
print(x_rem['text'][0])
print("Type:", type(x_rem['text'][0]))

My wife take birthday breakfast excellent The weather perfect make sit outside overlook ground absolute pleasure Our waitress excellent food arrive quickly semibusy Saturday morning It look like place fill pretty quickly earlier get better Do favor get Bloody Mary It phenomenal simply best I ever I pretty sure use ingredients garden blend fresh order It amaze While EVERYTHING menu look excellent I white shuffle scramble egg vegetable skilled tasty delicious It come piece riddle bread amaze absolutely make meal complete It best `` toast I ever Anyway I ca nt wait go back
Type: <class 'str'>


In [58]:
# Displays the dataframe after cleaning null values
x_rem

,text
0,My wife take birthday breakfast excellent The ...
1,I idea people give bad review place It go show...
2,love gro plate Rice good I also dig candy sele...
3,Rose Dakota I VE Chaparral Dog Park It conveni...
4,General Manager Scott Peter good egg Not go de...
...,...
9995,First visit Had lunch today use Group We order...
9996,Should call house deliciousness I could go ite...
9997,I recently visit Olive Ivy business last week ...
9998,My nephew move Scottsdale recently bunch frien...


### TFIDF

In [59]:
# This function ventorizes the data and returns 2 set of values
def tfidf_extractor(corpus):
  vectorizer = TfidfVectorizer()
  features = vectorizer.fit_transform(corpus)
  return vectorizer, features

In [60]:
# Calls the tfidf extractor function with the data as an argument
tfidf_vectorizer, tfidf_features = tfidf_extractor(x_rem['text'])

In [64]:
# Converts the feature results to dense
features = tfidf_features.todense()
print(features)
print("Type:", type(features))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Type: <class 'numpy.matrix'>


In [65]:
# Displays all the features extracted
feature_names = tfidf_vectorizer.get_feature_names()
print(feature_names)
print("Type:", type(feature_names))

['aa', 'aaa', 'aap', 'aaron', 'ab', 'ababa', 'abandon', 'abbey', 'abbrev', 'abbreviate', 'abbreviations', 'abdomen', 'abe', 'aberration', 'abide', 'abilities', 'ability', 'able', 'abnormally', 'abodoba', 'abogado', 'abound', 'about', 'aboutus', 'above', 'aboveaverage', 'abrasive', 'abreast', 'abridged', 'abroad', 'abrupt', 'abruptly', 'abs', 'absence', 'absent', 'absentee', 'absmag', 'absolute', 'absolutely', 'absorb', 'absorption', 'abstain', 'abstract', 'absurd', 'abt', 'abuelita', 'abuelo', 'abundance', 'abundant', 'abundantly', 'abuse', 'abusive', 'ac', 'aca', 'acad', 'academia', 'academie', 'academy', 'acapulco', 'accelerate', 'accent', 'accept', 'acceptable', 'access', 'accessibility', 'accessible', 'accessories', 'accessory', 'accident', 'accidental', 'accidentally', 'accidents', 'acclaim', 'acclimate', 'accolades', 'accommodate', 'accommodating', 'accommodation', 'accommodations', 'accompaniment', 'accompany', 'accomplice', 'accomplish', 'accomplishment', 'accomplishments', 'ac

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [66]:
# This function creates a dataframe with the features as values
def display_features(features, feature_names):
    df = pd.DataFrame(data=features, columns=feature_names)
    print(df)

In [67]:
# Calls the display features function for dataframe conversion
display_features(features, feature_names)
print("Type:", type(display_features))

       aa  aaa  aap  aaron   ab  ababa  abandon  abbey  abbrev  abbreviate  \
0     0.0  0.0  0.0    0.0  0.0    0.0      0.0    0.0     0.0         0.0   
1     0.0  0.0  0.0    0.0  0.0    0.0      0.0    0.0     0.0         0.0   
2     0.0  0.0  0.0    0.0  0.0    0.0      0.0    0.0     0.0         0.0   
3     0.0  0.0  0.0    0.0  0.0    0.0      0.0    0.0     0.0         0.0   
4     0.0  0.0  0.0    0.0  0.0    0.0      0.0    0.0     0.0         0.0   
...   ...  ...  ...    ...  ...    ...      ...    ...     ...         ...   
9995  0.0  0.0  0.0    0.0  0.0    0.0      0.0    0.0     0.0         0.0   
9996  0.0  0.0  0.0    0.0  0.0    0.0      0.0    0.0     0.0         0.0   
9997  0.0  0.0  0.0    0.0  0.0    0.0      0.0    0.0     0.0         0.0   
9998  0.0  0.0  0.0    0.0  0.0    0.0      0.0    0.0     0.0         0.0   
9999  0.0  0.0  0.0    0.0  0.0    0.0      0.0    0.0     0.0         0.0   

      ...  zuma  zumaroka  zur  zweig  zwiebelkräuser   zz  écl

## 5) Creating the model

### -Define x and y matrix. x as the text, y as the labels

In [101]:
x = x_rem['text'].values
y = y[['cool', 'useful', 'funny']].values
print("Type for x:", type(x))
print("Type for y:", type(y))

Type for x: <class 'numpy.ndarray'>
Type for y: <class 'numpy.ndarray'>


### -Perform train-test-split using a test size of 0.25, random state of 101

In [104]:
sentences_train, sentences_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=101)
print("Shape of sentences_train:  ", sentences_train.shape)
print("Shape of sentences_test:   ", sentences_test.shape)
print("Shape of y_train:          ", y_train.shape)
print("Shape of y_test:           ", y_test.shape)

Shape of sentences_train:   (7500,)
Shape of sentences_test:    (2500,)
Shape of y_train:           (7500, 3)
Shape of y_test:            (2500, 3)


### -Creating the Logistic Regression model
### -with LabelPowerset for multi-labeled datasets

In [81]:
vectorizer = TfidfVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test = vectorizer.transform(sentences_test)

In [82]:
classifier = LabelPowerset(LogisticRegression(max_iter=10000))

## 6) Training and testing

In [84]:
classifier.fit(X_train, y_train)

LabelPowerset(classifier=LogisticRegression(max_iter=10000),
              require_dense=[True, True])

In [97]:
y_predicted = classifier.predict(X_test)

In [105]:
print("Type: for y_test:      ", type(y_test))
print("Type for y_predicted:  ", type(y_predicted))

Type: for y_test:       <class 'numpy.ndarray'>
Type for y_predicted:   <class 'scipy.sparse.lil.lil_matrix'>


In [106]:
testing = np.argmax(y_test, axis=1)

y_predicted = y_predicted.toarray()

y_test = np.argmax(y_test, axis=1)
y_pred = np.argmax(y_predicted, axis=1)

In [107]:
print("Type for y_pred: ", type(y_test))
print("Type for y_test: ", type(y_pred))

Type for y_pred:  <class 'numpy.ndarray'>
Type for y_test:  <class 'numpy.ndarray'>


## 7) Performance evaluation

In [110]:
conf = confusion_matrix(y_test, y_pred)
print("Type:", type(conf))

Type: <class 'numpy.ndarray'>


In [111]:
tp = conf[0][0]
fn = conf[0][1]
fp = conf[1][0]
tn = conf[1][1]

In [113]:
precision = tp/(tp+fp)
accuracy = (tp+tn)/(tp+fn+fp+tn)
recall = tp/(tp+fn)
f1 = 2*(precision*recall)/(precision+recall)
lrpre, lracc, lrrec, lrf1 = precision, accuracy, recall, f1
print("Precision: ", lrpre)
print("Accuracy: ", lracc)
print("Recall: ", lrrec)
print("F1-Score: ", lrf1)

Precision:  0.5960637300843487
Accuracy:  0.5868544600938967
Recall:  0.9230769230769231
F1-Score:  0.7243735763097949


### -Display Confusion Matrix

In [114]:
results = [("LR", lrpre, lracc, lrrec, lrf1)]
          
pd.options.display.float_format = '{:,.2f}'.format
dtable1 = pd.DataFrame(results, columns = ['MODEL', 'Precision', 'Accuracy', 'Recall', 'F1-Score'])
# dtable1.sort_values(by=['Accuracy'], ascending=False)
dtable1

,MODEL,Precision,Accuracy,Recall,F1-Score
0,LR,0.60,0.59,0.92,0.72
